In [1]:
from numpy import save, load
from pandas import read_csv
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import keras.layers 
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets
from numpy import save, load
from pandas import read_csv
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense,Activation
from keras.models import Input, Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,TimeDistributed, MaxPooling1D
import tensorflow as tf
import numpy as np
import keras.backend as K
import keras.layers
from keras import optimizers
from keras.layers import Activation, Lambda
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from typing import List, Tuple 
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/MyDrive/Colab Notebooks/Dataset1/trainX.npy')
y_train = np.load('gdrive/My Drive/Colab Notebooks/Dataset1/trainy.npy')
x_test = np.load('gdrive/My Drive/Colab Notebooks/Dataset1/testX.npy')
y_test = np.load('gdrive/My Drive/Colab Notebooks/Dataset1/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/gdrive
(33104, 128, 3) (33104, 118) (3740, 128, 3) (3740, 118)


In [3]:
x_train = x_train.transpose(0,2,1)
x_test = x_test.transpose(0,2,1)

inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)
#from sklearn.model_selection import train_test_split
#x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [4]:
x_train.shape

(33104, 3, 128)

In [7]:
y_train = (np.argmax(y_train, axis=1)).reshape(-1, 1)
y_test =  (np.argmax(y_test, axis=1)).reshape(-1, 1)

In [8]:
y_train.shape

(33104, 1)

In [5]:
embed_dim = x_train.shape[-1]   # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = x_train.shape[-1]  # Hidden layer size in feed forward network inside transformer

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads,
                                             key_dim=embed_dim)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)  # self-attention layer
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)  # layer norm
        ffn_output = self.ffn(out1)  #feed-forward layer
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)  # layer norm


In [10]:
acc_per_fold = []
loss_per_fold = []
num_folds = 2
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)
skf = StratifiedKFold(n_splits=5, shuffle=True)
fold_no = 1
input_shape = x_train.shape[1:]
input = Input(shape=x_train.shape[1:])
for train, test in skf.split(inputs, targets):
    model = keras.Sequential()
    model.add(layers.Input(shape= x_train.shape[1:]))
    model.add(TransformerBlock(embed_dim, num_heads, ff_dim))
    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(ff_dim, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(118, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(inputs[train], targets[train], epochs=200, verbose=True, batch_size=128,callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=5,restore_best_weights=True)])
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

  # Increase fold number
    fold_no = fold_no + 1
model.summary()
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

Epoch 1/200
231/231 [==============================] - 4s 11ms/step - loss: 2.4168 - accuracy: 0.4631
Epoch 2/200
231/231 [==============================] - 3s 11ms/step - loss: 1.0636 - accuracy: 0.7477
Epoch 3/200
231/231 [==============================] - 3s 11ms/step - loss: 0.7762 - accuracy: 0.8107
Epoch 4/200
231/231 [==============================] - 2s 11ms/step - loss: 0.6374 - accuracy: 0.8379
Epoch 5/200
231/231 [==============================] - 3s 11ms/step - loss: 0.5534 - accuracy: 0.8577
Epoch 6/200
231/231 [==============================] - 3s 11ms/step - loss: 0.4829 - accuracy: 0.8740
Epoch 7/200
231/231 [==============================] - 3s 11ms/step - loss: 0.4443 - accuracy: 0.8812
Epoch 8/200
231/231 [==============================] - 3s 11ms/step - loss: 0.4084 - accuracy: 0.8900
Epoch 9/200
231/231 [==============================] - 3s 11ms/step - loss: 0.3762 - accuracy: 0.8960
Epoch 10/200
231/231 [==============================] - 3s 11ms/step - loss: 0.355

In [11]:
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.4109150171279907 - Accuracy: 92.0341968536377%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.38824936747550964 - Accuracy: 92.40059852600098%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.4437847137451172 - Accuracy: 92.35988855361938%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.38448891043663025 - Accuracy: 92.50916242599487%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.35381293296813965 - Accuracy: 92.90173649787903%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 92.44111657142639 (+- 0.2795082519627537)
> Loss: 0.3962501883506775
--------------------------